<a href="https://colab.research.google.com/github/Justin21523/edge-deid-studio/blob/feature%2Fadd-gpt2-inference-notebook/notebooks/04_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在 `04_inference.ipynb` 中，我們的目標是從剛才微調好的 GPT-2 模型讀取檔案，接收一段文字 prompt，然後產生對應的後續文字。整體流程跟前面微調的 notebook 類似：先做環境設定、再載入模型、然後寫一個生成（inference）函式，最後示範幾個 prompt 的結果。

## 一、環境準備（Cell 1–4）

In [ ]:
!pip install -q transformers accelerate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['HF_HOME'] = '/content/drive/MyDrive/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/hf_cache/transformers'
os.makedirs(os.environ['TRANSFORMERS_CACHE'], exist_ok=True)

In [ ]:
from huggingface_hub import login
from getpass import getpass
hf_token = getpass("請貼上你的 Hugging Face token：")
login(token=hf_token)

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())

這一段跟 `03_finetune_gpt2.ipynb` 完全一致：安裝 Transformers、Accelerate，設定 Drive 快取路徑、登入 Hugging Face、檢查 GPU。先確認程式碼能跑過，等到啟用 GPU 時直接使用。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_dir = "models/gpt2/v1.0"   # 與微調時的 output_dir 對應
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForCausalLM.from_pretrained(model_dir)
model.eval()                     # 推理模式，關閉 dropout

這裡我們把 `AutoTokenizer` 和 `AutoModelForCausalLM` 都指向剛才儲存的資料夾 `models/gpt2/v1.0`，確保讀到的是你微調之後的版本。呼叫 `model.eval()` 會關閉訓練專用的 dropout 等機制，使推理結果更穩定。

## 三、定義推理函式（Cell 6）

In [ ]:
import torch

def generate_text(prompt: str,
                  max_length: int = 100,
                  temperature: float = 1.0,
                  top_k: int = 50,
                  top_p: float = 0.95,
                  repetition_penalty: float = 1.2):
    """
    接收一段 prompt，使用 sampling 方式產生後續文字。
    - max_length: 包含 prompt 的最大 token 長度
    - temperature: 控制 sampling 隨機性，越高越多樣
    - top_k: 只從機率最高的 k 個 token 中取樣
    - top_p: 累積機率截斷 (nucleus sampling)
    - repetition_penalty: 懲罰重複字詞
    """
    # 1. 將文字轉為 token id tensor
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # 2. 產生 (此行待啟用 GPU 時取消註解)
    # outputs = model.generate(
    #     input_ids=input_ids.to(model.device),
    #     attention_mask=attention_mask.to(model.device),
    #     max_length=max_length,
    #     temperature=temperature,
    #     top_k=top_k,
    #     top_p=top_p,
    #     repetition_penalty=repetition_penalty,
    #     do_sample=True,
    #     pad_token_id=tokenizer.eos_token_id
    # )
    # generated = outputs[0]

    # 3. 將生成結果轉回文字
    # return tokenizer.decode(generated, skip_special_tokens=True)

    # 目前先回傳參數確認
    return dict(
        prompt=prompt,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )

這個函式裡，我們解釋了每個生成參數的意義，並且把真正呼叫 `model.generate()` 的部分先註解，以便你先跑語法檢查、再等 GPU 環境時再開啟。`do_sample=True` 表示採樣而非 beam search，適合需要多樣化輸出的場景。

## 四、執行推理示範（Cell 7）

In [ ]:
# 範例 1：短 prompt
result1 = generate_text("In a distant future, humanity has", max_length=80)
print(result1)

# 範例 2：敘事風格提示
result2 = generate_text("Once upon a time in a small village,", max_length=120, temperature=0.8, top_p=0.9)
print(result2)

這裡我們先呼叫剛剛的函式，輸出的是參數確認的字典。等到把真正的 `generate()` 打開之後，執行結果就會是真正生成的文本。這一步驟主要是讓你檢查程式碼能不能順利執行。

In [ ]:
## 五、結果儲存與解讀（Cell 8）

In [ ]:
# 真正啟用 GPU 之後，可以把下面這段解開註解，並把結果寫到檔案或 Colab 上
# with open("inference_results.txt", "w") as f:
#     for prompt in ["In a distant future, humanity has", "Once upon a time in a small village,"]:
#         text = generate_text(prompt, max_length=100, temperature=1.0)
#         f.write(text + "\n\n")

# 讀回並顯示結果
# !head -n 20 inference_results.txt

當切換到有 GPU 的 Colab Pro，就可以把寫檔與讀檔的程式碼註解解除，自動把多個 prompt 的生成結果儲存到 `inference_results.txt`，方便後續分析或分享。

在自回歸 (causal) 語言模型中，`model.generate()` 會依序預測下一個 token 的機率分布，然後根據我們設定的 sampling 策略（如 temperature、top-k、top-p）抽樣一個 token，接著把它接回輸入，繼續預測下一個。這樣一步步往前，直到達到 `max_length` 或遇到 EOS token 為止。`repetition_penalty` 用於抑制重複文字，讓生成更自然。